<a href="https://colab.research.google.com/github/TalissaMoura/AmazonC4DatasetAugmented/blob/main/notebooks/2_pipeline_for_calculate_the_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Pipeline for measure the metrics for the hard queries generated

# 2.1 - Imports



In [1]:
!pip install outlines

In [2]:
import os, json, re, torch
import outlines
from huggingface_hub import login,snapshot_download, upload_file, HfApi,hf_hub_download
from huggingface_hub.hf_api import HfApi
from huggingface_hub.errors import EntryNotFoundError
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from google.colab import userdata
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from pydantic import BaseModel,ValidationError
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 2.2 - Functions

In [41]:
class Constraints(BaseModel):
    constraints: list[str]
class Intents(BaseModel):
    intents: list[str]


def extract_json_from_markdown(response_text: str) -> str:
    """Extract JSON from markdown code blocks"""
    # Look for ```json ... ``` or ``` ... ```
    pattern = r'```(?:json)?\s*(.*?)\s*```'
    matches = re.findall(pattern, response_text, re.DOTALL)

    if matches:
        # Return the first JSON block found
        return matches[0].strip()
    else:
        # If no markdown blocks, return original text
        return response_text

def clean_and_parse_json(response_text: str,pydantic_model:BaseModel) -> BaseModel | None:
    # Step 1: Extract JSON from markdown if present
    json_content = extract_json_from_markdown(response_text)

    # Step 2: Remove any remaining backticks or markdown artifacts
    json_content = json_content.strip()

    # Step 3: Handle potential escaping issues
    if json_content.startswith('"') and json_content.endswith('"'):
        # Unescape if it's still a string representation
        try:
            json_content = json.loads(json_content)
        except json.JSONDecodeError:
            pass  # Keep as is if it's not double-escaped

    # Step 4: Parse and validate
    try:
        if isinstance(json_content, str):
            return pydantic_model.model_validate_json(json_content)
        else:
            return pydantic_model.model_validate(json_content)
    except ValidationError as e:
        print(f"Validation error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None

from pydantic import ValidationError
import time

def generate_and_parse_with_retry(
    model,
    tokenizer,
    messages,
    pydantic_model,
    max_attempts=3,
    delay=0.5  # optional: to avoid overwhelming GPU or API rate limits
):
    for attempt in range(1, max_attempts + 1):
        # Tokenize and generate
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048
        )
        # Trim input part
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(response)

        # Try parsing
        try:
            parsed = clean_and_parse_json(response, pydantic_model)
            print(parsed)
            if parsed is not None:
                return parsed  # success!
        except ValidationError:
            pass  # will retry

        # Optional: small delay before next attempt
        if attempt < max_attempts:
            time.sleep(delay)

    # All attempts failed
    return None  # or raise an exception, depending on desired behavior


def generate_constraint_list(df_queries,
                             model,
                             tokenizer,
                             category,
                             pydantic_model=BaseModel,
                             system_template_path=None,
                             user_template_path=None,
                             save_local=True):
  user_template = outlines.Template.from_file(f"{user_template_path}")
  system_template = outlines.Template.from_file(f"{system_template_path}")
  batches = df_queries.to_batches()
  for i, batch in enumerate(batches):
      new_rows = []
      for row in batch.to_pylist():
          query = row['hard_negative_query']

          messages = [
              {"role": "system", "content": system_template()},
              {"role": "user", "content": user_template(user_query=query)}
          ]

          # Try up to 3 times to get a valid parse
          response_treated = generate_and_parse_with_retry(
              model=model,
              tokenizer=tokenizer,
              messages=messages,
              pydantic_model=pydantic_model,
              max_attempts=3
          )

          if response_treated:
            new_rows.append(response_treated)
          else:
            new_rows.append([""])


  batch_pylist = batch.to_pylist()
  new_data = []
  for new_row,data in zip(new_rows,batch_pylist):
      data["constraints"] = new_row.__dict__["constraints"]
      new_data.append(data)

  if save_local:
    batch = pa.Table.from_pylist(new_data)
    base = pathlib.Path("/content/data/datasets/procesed/contraint_lists_for_generated_queries")
    if not base.exists():
      base.mkdir(parents=True)
    pq.write_table(batch, f"/content/data/datasets/procesed/contraint_lists_for_generated_queries/{category}_part_{i}.parquet")
    print(f"Batch {i} done.")


def generate_intent_list(df_queries,
                         model,
                         tokenizer,
                         category,
                         pydantic_model=BaseModel,
                         text_type="review",
                         system_template_path=None,
                         user_template_path=None,
                         save_local=True):

  user_template = outlines.Template.from_file(f"{user_template_path}")
  system_template = outlines.Template.from_file(f"{system_template_path}")
  batches = df_queries.to_batches()
  for i, batch in enumerate(batches):
      new_rows = []
      for row in batch.to_pylist():
          if text_type == "review":
            text = row['ori_semi_positive_review']
          else:
            text = row['hard_negative_query']
          messages = [
              {"role": "system", "content": system_template()},
              {"role": "user", "content": user_template(text=text,text_type=text_type)}
          ]

          response_treated = generate_and_parse_with_retry(
              model=model,
              tokenizer=tokenizer,
              messages=messages,
              pydantic_model=pydantic_model,
              max_attempts=3
          )

          if response_treated:
              new_rows.append({
                  "text_type":"review",
                   "intents":response_treated.__dict__["intents"]
              })
          else:
              new_rows.append(
                  {
                  "text_type":"search_query",
                   "intents":response_treated.__dict__["intents"]
              })



  batch_pylist = batch.to_pylist()
  new_data = []
  for new_row,data in zip(new_rows,batch_pylist):
    if text_type == "review":
      data["user_review_intent"] = new_row["intents"]
    else:
      data["user_query_intent"] = new_row["intents"]
    new_data.append(data)

  if save_local:
    batch = pa.Table.from_pylist(new_data)
    base = pathlib.Path(f"/content/data/datasets/procesed/user_intent/{text_type}")
    if not base.exists():
      base.mkdir(parents=True)
    pq.write_table(batch, f"/content/data/datasets/procesed/user_intent/{text_type}/{category}_part_{i}.parquet")
    print(f"Batch {i} done.")


def load_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True,device_map="cpu",)
  model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16,device_map="auto")

  # 🔍 Validate placement
  if not next(model.parameters()).is_cuda:
      raise RuntimeError("Model failed to load on GPU!")
  print("✅ Model successfully on GPU.")
  return model,tokenizer


# 2.3 - Load Data

In [43]:
from google.colab import userdata
from datasets import load_dataset
import pathlib
hf_token = userdata.get('HF_TOKEN')
df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
def download_hard_queries_datasets_per_category(idx_category):
  try:
    base = pathlib.Path("/content/data/datasets/procesed/hard_negative_queries")
    if not base.exists():
      base.mkdir(parents=True)
    files = list(base.glob(f"{idx_category}_part_*.parquet"))
    if len(files) == 0:
      raise FileNotFoundError
    return f"List of files for {idx_category}: {files}"
  except FileNotFoundError:
    snapshot_download(
        repo_id='Talissa/AmazonC4Augmented',
        repo_type='dataset',
        local_dir="/content/data",
        allow_patterns=[f'datasets/procesed/hard_negative_queries/{idx_category}_part_*.parquet'],
        token=hf_token,
    )
    return f"Downloaded hard negative files for category {idx_category}"

def download_json_metrics_per_category(idx_category):
    try:
      hf_hub_download(
          repo_id='Talissa/AmazonC4Augmented',
          repo_type='dataset',
          local_dir="/content/data",
          filename="/metrics/quality/quality_metrics_per_category.json",
          token=hf_token,
      )
    except EntryNotFoundError:
      quality_metrics_path = pathlib.Path("/content/data/datasets/metrics/quality")
      if not quality_metrics_path.exists():
        quality_metrics_path.mkdir(parents=True)
        with open(f"/content/data/datasets/metrics/quality/quality_metrics_per_category.json","w+") as f:
          json.dump([{}],f,indent=2)
          f.close()

      file = json.load(open(f"/content/data/datasets/metrics/quality/quality_metrics_per_category.json"))
      return file

# setup hugging faces
hf_api = HfApi(token=hf_token)
## Download the datasets
idx_category = "All_Beauty"
download_hard_queries_datasets_per_category(idx_category)
## Download json
dict_quality_metrics = download_json_metrics_per_category(idx_category)
print(dict_quality_metrics)

[{}]


1 - Calculate similarity scores

In [44]:
if "model_eval" not in globals():
  model_eval = SentenceTransformer('all-MiniLM-L6-v2',device="cuda")
else:
  print("Model eval already loaded")

def add_queries_similarity_score(query,hard_negative_query):
  if not query or not hard_negative_query:
    return 0.0
  query_emb = model_eval.encode(query, convert_to_tensor=False)
  hard_negative_query_emb = model_eval.encode(hard_negative_query, convert_to_tensor=False)

  return cosine_similarity(query_emb, hard_negative_query_emb)


# Filter amazonc4 dataset for the items_ids present in all_Beauty dataset
dict_quality_scores_category = {}
df_hq = pq.read_table(f'/content/data/datasets/procesed/hard_negative_queries/{idx_category}_part_0.parquet')
df_queries = df_amazonc4.select(["item_id","query"]).join(
    df_hq.select(["item_id","hard_negative_query"]),
    keys=["item_id"],
    join_type="inner",
)

df_queries = df_queries.sort_by("item_id")
queries_list = df_queries.to_pylist()
sim_list = []
for query in queries_list:
  sim = add_queries_similarity_score([query["query"]],[query["hard_negative_query"]])
  sim_list.append(sim)

print("Average sim score for queries and hard negative queires: ", np.array(sim_list).mean())

if idx_category not in dict_quality_scores_category:
  dict_quality_scores_category[idx_category] = {"hard_queries_and_queries_sim_score":np.array(sim_list).mean()}
else:
  dict_quality_scores_category[idx_category]["hard_queries_and_queries_sim_score"] = np.array(sim_list).mean()

Model eval already loaded
Average sim score for queries and hard negative queires:  0.6658821


2 - Calculate the Constraint Preservation Rate

In [45]:
base = pathlib.Path("/content/data/datasets/procesed/hard_negative_queries")
files = list(base.glob(f"{idx_category}_part_*.parquet"))
print(f'Current local dir files for {idx_category}: {files}')
dfs_reviews = []
for data_file in files:
  df = pq.read_table(f'{data_file}')
  if "ori_semi_positive_review_list" in df.column_names:
    df = df.rename_columns({"ori_semi_positive_review_list":"ori_semi_positive_review"})
  if "ori_positive_review_distinct" in df.column_names:
    df = df.rename_columns({"ori_positive_review_distinct":"ori_positive_review"})
  print(f"Loaded {data_file}")
  dfs_reviews.append(df)
df_reviews_all = pa.concat_tables(dfs_reviews)

print(f"loaded {df_reviews_all.num_rows} rows for {idx_category}")

Current local dir files for All_Beauty: [PosixPath('/content/data/datasets/procesed/hard_negative_queries/All_Beauty_part_0.parquet')]
Loaded /content/data/datasets/procesed/hard_negative_queries/All_Beauty_part_0.parquet
loaded 27 rows for All_Beauty


In [46]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/system_prompt_generate_query_restritions_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """

    ```json
      {
        "task": "You are a constraint extractor. Given a product search query, identify and list *only* the explicit or strongly implied characteristics that the user wants to *exclude* or *avoid* in the product. Do not infer preferences unless clearly stated (e.g., 'no plastic', 'avoid heavy items'). Return an empty array if no such constraints exist.",
        "input_format": {
          "query": "string - the user's natural-language product search query",
          "text_type": "string - review or search query"
        },
        "output_type": "json",
        "output_format": {
          "constraints": [
            "string — a phrase that summarize what the user don't want in the product (i.e this product should not be small, this product should avoid the brand A, the user don't want overprice products .. )",
            "Avoid vague terms; prefer concrete, actionable constraints derived directly from the query."
          ]
        },
        "requirements": [
          "Constraints must be phrased as *what to avoid*, not what is desired.",
          "Normalize phrasing (e.g., use 'corded' instead of 'with a cord' if excluding cordless devices).",
          "Do NOT include positive requirements (e.g., 'durable' → ignore unless phrased negatively like 'not fragile').",
          "If ambiguity exists, prefer under-inclusion (i.e., only include unambiguous exclusions)."
        ]
      }
    ```

    """

    with open('/content/prompts/system_prompt_generate_query_restritions_list.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/system_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for constraint preservation rate"
          )

No files have been modified since last commit. Skipping to prevent empty commit.


In [47]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_generate_query_restritions_list"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    ```json
        {

          "query": {{user_query}}}

        }
    ```
    """

    with open('/content/prompts/user_prompt_generate_query_restritions_list.txt', 'w') as f:
          f.write(user_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/user_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for constraint preservation rate"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [48]:
print('-- generate constraint list --')
MODEL_NAME = 'Qwen/Qwen2.5-3B-Instruct'
if "model" not in globals():
  model, tokenizer = load_model(MODEL_NAME)
else:
  print("Model already loaded")
generate_constraint_list(df_reviews_all,
                         model,tokenizer,
                         category=idx_category,
                         pydantic_model=Constraints,
                         system_template_path='/content/prompts/system_prompt_generate_query_restritions_list.txt',
                         user_template_path='/content/prompts/user_prompt_generate_query_restritions_list.txt',
                         )

-- generate constraint list --
Model already loaded
```json
{
  "constraints": [
    "tiny bit small",
    "slightly too tingly"
  ]
}
```
constraints=['tiny bit small', 'slightly too tingly']
```json
{
  "constraints": [
    "tiny bit small",
    "slightly too tingly"
  ]
}
```
constraints=['tiny bit small', 'slightly too tingly']
```json
{
  "constraints": [
    "tiny bit small",
    "tingling for 20 minutes"
  ]
}
```
constraints=['tiny bit small', 'tingling for 20 minutes']
```json
{
  "constraints": [
    "avoid takes a while to use",
    "avoid uncomfortable on teeth"
  ]
}
```
constraints=['avoid takes a while to use', 'avoid uncomfortable on teeth']
```json
{
  "constraints": [
    "comfortable",
    "not uncomfortable",
    "doesn’t cause sensitivity"
  ]
}
```
constraints=['comfortable', 'not uncomfortable', 'doesn’t cause sensitivity']
```json
{
  "constraints": [
    "difficult to control",
    "took a while to use",
    "nervousness"
  ]
}
```
constraints=['difficult to co

In [49]:
def compute_consistent_score(x, y):
    # Handle edge cases
    if not x or not y:
        return 0.0  # or np.nan — up to you

    # Encode all sentences
    X = model_eval.encode(x, convert_to_tensor=False)
    Y = model_eval.encode(y, convert_to_tensor=False)

    # Compute cosine similarity matrix: shape = (len(X), len(Y))
    sims = cosine_similarity(X,Y)

    # For each constraint, get max similarity over negative embeddings
    max_sims = sims.max(axis=1)  # shape: (len(X),)

    # Return the biggest value over the similarities
    return float(np.max(max_sims))

def calculate_consistent_score(df_reviews):
  new_rows = []
  for batch in df_reviews.to_batches():
    for row in batch.to_pylist():
      negative_points = row['negative_points']
      constraint_list = row['constraints']
      row["consistent_score"] = compute_consistent_score(constraint_list,negative_points)
      row['consistent_hit'] = 1.0 if row['consistent_score'] >= 0.6 else 0.0
      new_rows.append(row)
  new_batch = pa.Table.from_pylist(new_rows)
  return new_batch

base_reviews_processed = pathlib.Path("/content/data/datasets/procesed/contraint_lists_for_generated_queries")
files = list(base_reviews_processed.glob(f"{idx_category}_part_*.parquet"))
df_reviews_processed = []
for data_file in files:
  df_review_processed = pq.read_table(f'{data_file}')
  df_reviews_processed.append(df_review_processed)
  print(f"Loaded {data_file}")
df_reviews_w_metrics = calculate_consistent_score(pa.concat_tables(df_reviews_processed))

Loaded /content/data/datasets/procesed/contraint_lists_for_generated_queries/All_Beauty_part_0.parquet


3 - Check the user intent score

In [50]:
import outlines, pathlib

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/system_prompt_generate_user_intent_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    ```json
         {
        "task": "You are a user intent extractor. You recieve a text from a user that describes a product they review or a search query. Your task is to identify **what** the user wants from the product. — i.e., what the user fundamentally expects, requires, or rejects in a product.",
        "instructions": [
        "1. Identify **only explicit or strongly implied intents** — do NOT infer preferences beyond the text.",
        "2. For each expectation or requirement identified, create an intent object with the following structure:",
        "   - `intent`: a concise, canonical phrase describing the user's expectation (noun-phrase preferred, e.g., 'good grip on wet surfaces').",
        "3. Prioritize *avoidance* and *constraint* intents — they are critical for query rewriting and hard negative generation.",
        "4. The input type is describe by the text_type parameter. The input can be a review or a search query",
        "5. If the input is a review, distinguish between:",
        "   - Desired traits (what worked or was expected)",
        "   - Undesired traits (what failed or must be avoided in next purchase)."
        ],

        "input_format": {
          "text": "string — the user's natural-language product search query",
          "text_type": "string - review or search query"
        },
        "output_type": "json",
        "output_format": {
          "intents": [
            "string — a text that summarizes what the user expects from the product. ",
            "Avoid vague terms; prefer concrete, actionable constraints derived directly from the query."
          ]
        },
        "requirements": [
        "Return only the JSON format as specified; do not include any additional text or explanations.",
        "Do not include vague intents like 'good quality' or 'reliable' unless explicitly defined in the text.",
        "Rephrase avoidance intents as *positive requirements* where possible (e.g., 'no slipping' → 'non-slip grip').",
        "Use **English** for `intent` field (to improve cross-lingual embedding consistency), even if input is in Portuguese.",
        "Maximum 6 intents. Prefer specificity over exhaustiveness."
    ]
      }

    ```
    """

    with open('/content/prompts/system_prompt_generate_user_intent_list.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_user_intent_list.txt",
              path_in_repo=f"prompts/system_prompt_generate_user_intent_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for user intent score rate"
          )

No files have been modified since last commit. Skipping to prevent empty commit.


In [51]:
import outlines, pathlib

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/user_prompt_generate_user_intent_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """

    ```json
    {
      text: {{text}},
      text_type: {{text_type}}
    }
    ```

    """

    with open('/content/prompts/user_prompt_generate_user_intent_list.txt', 'w+') as f:
          f.write(user_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_user_intent_list.txt",
              path_in_repo=f"prompts/user_prompt_generate_user_intent_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for user intent score rate"
          )

No files have been modified since last commit. Skipping to prevent empty commit.


In [52]:
# generate user intent from query
generate_intent_list(
    df_reviews_w_metrics,
    model,
    tokenizer,
    category=idx_category,
    pydantic_model=Intents,
    system_template_path='/content/prompts/system_prompt_generate_user_intent_list.txt',
    user_template_path='/content/prompts/user_prompt_generate_user_intent_list.txt',
    text_type="review",
)


```json
{
  "intents": [
    "hydrated and moisturized skin",
    "fits well",
    "environmentally friendly",
    "no strong tingling sensations",
    "enough serum to hydrate",
    "long-lasting without irritation"
  ]
}
```
intents=['hydrated and moisturized skin', 'fits well', 'environmentally friendly', 'no strong tingling sensations', 'enough serum to hydrate', 'long-lasting without irritation']
```json
{
  "intents": [
    "hydrated and moisturized skin",
    "comfortable fit",
    "environmentally friendly packaging",
    "no strong tingling sensation",
    "enough serum content",
    "suitable for morning use"
  ]
}
```
intents=['hydrated and moisturized skin', 'comfortable fit', 'environmentally friendly packaging', 'no strong tingling sensation', 'enough serum content', 'suitable for morning use']
```json
{
  "intents": [
    "hydrated and moisturized skin",
    "fits well",
    "environmentally friendly material",
    "non-tingly experience",
    "not too small",
    "enoug

In [53]:
# generate user review intent from query
generate_intent_list(
    df_reviews_w_metrics,
    model,
    tokenizer,
    category=idx_category,
    pydantic_model=Intents,
    system_template_path='/content/prompts/system_prompt_generate_user_intent_list.txt',
    user_template_path='/content/prompts/user_prompt_generate_user_intent_list.txt',
    text_type="search_query",
)

```json
{
  "intents": [
    "hydrating and soothed skin",
    "large size compared to other brands",
    "no excessive tingling sensation"
  ]
}
```
intents=['hydrating and soothed skin', 'large size compared to other brands', 'no excessive tingling sensation']
```json
{
  "intents": [
    "hydrated and soothed skin after use",
    "large size compared to other brands",
    "no excessive tingling sensation"
  ]
}
```
intents=['hydrated and soothed skin after use', 'large size compared to other brands', 'no excessive tingling sensation']
```json
{
  "intents": [
    "hydrated and soothed skin",
    "not too small",
    "no tingling after use"
  ]
}
```
intents=['hydrated and soothed skin', 'not too small', 'no tingling after use']
```json
{
  "intents": [
    "non-slip grip",
    "quick usage",
    "comfortable on teeth",
    "effective plaque control",
    "short handling time",
    "reduces anxiety"
  ]
}
```
intents=['non-slip grip', 'quick usage', 'comfortable on teeth', 'effective

In [54]:
def calculate_intent_consistent_score(df_reviews):
  new_rows = []
  for batch in df_reviews.to_batches():
    for row in batch.to_pylist():
      user_review_intent = row['user_review_intent']
      user_query_intent = row['user_query_intent']
      row["user_intent_score"] = compute_consistent_score(user_query_intent,user_review_intent)
      new_rows.append(row)
  new_batch = pa.Table.from_pylist(new_rows)
  return new_batch

## Search for files for user intent based on reviews
base_intent_review = pathlib.Path("/content/data/datasets/procesed/user_intent/review")
files_intent_review = list(base_intent_review.glob(f"{idx_category}_part_*.parquet"))
print(f'Current local dir files for {idx_category}: {files_intent_review}')
dfs_reviews_intent_review = []
for data_file in files_intent_review:
  df_reviews_intent_review = pq.read_table(f'{data_file}')
  print(f"Loaded {data_file}")
  dfs_reviews_intent_review.append(df_reviews_intent_review)

## Search for files for user intent based on search query
base_intent_query = pathlib.Path("/content/data/datasets/procesed/user_intent/search_query")
files_intent_user = list(base_intent_query.glob(f"{idx_category}_part_*.parquet"))
print(f'Current local dir files for {idx_category}: {files_intent_user}')
dfs_reviews_intent_users = []
for data_file in files_intent_user:
  df_reviews_intent_user = pq.read_table(f'{data_file}')
  print(f"Loaded {data_file}")
  dfs_reviews_intent_users.append(df_reviews_intent_user)

df_raw_reviews_intent = pa.concat_tables(dfs_reviews_intent_review)
df_raw_user_intent = pa.concat_tables(dfs_reviews_intent_users)

df_raw_reviews_intent = df_raw_reviews_intent.sort_by("item_id")
df_raw_user_intent = df_raw_user_intent.sort_by("item_id")

df_reviews_w_metrics = df_raw_reviews_intent.append_column("user_query_intent",df_raw_user_intent.column("user_query_intent"))
df_reviews_w_metrics = calculate_intent_consistent_score(df_reviews_w_metrics)
print(df_reviews_w_metrics)

Current local dir files for All_Beauty: [PosixPath('/content/data/datasets/procesed/user_intent/review/All_Beauty_part_0.parquet')]
Loaded /content/data/datasets/procesed/user_intent/review/All_Beauty_part_0.parquet
Current local dir files for All_Beauty: [PosixPath('/content/data/datasets/procesed/user_intent/search_query/All_Beauty_part_0.parquet')]
Loaded /content/data/datasets/procesed/user_intent/search_query/All_Beauty_part_0.parquet
pyarrow.Table
item_id: string
ori_semi_positive_review: list<item: string>
  child 0, item: string
ori_positive_review: list<item: string>
  child 0, item: string
positive_points: list<item: string>
  child 0, item: string
negative_points: list<item: string>
  child 0, item: string
hard_negative_query: string
constraints: list<item: string>
  child 0, item: string
consistent_score: double
consistent_hit: double
user_review_intent: list<item: string>
  child 0, item: string
user_query_intent: list<item: string>
  child 0, item: string
user_intent_scor

4 - Save metrics and prompts in Hugging Faces

In [55]:
print(f"save metrics for {idx_category}")
metrics = ["consistent_score","consistent_hit","user_intent_score"]
for metric in metrics:
  if metric not in dict_quality_scores_category[idx_category]:
    calculate_result = df_reviews_w_metrics.group_by(["item_id"]).aggregate([(f"{metric}","mean")])
    result = calculate_result.to_pylist()[0][f"{metric}_mean"]
    dict_quality_scores_category[idx_category][metric] = result

## Add metrics into dict
search = idx_category
for cat in dict_quality_metrics:
  cursor = cat["category"] if cat != {} else -1
  if cursor == -1: ## Add first record
    cat = {
        "category":idx_category,
        "hard_queries_and_queries_sim_score":float(dict_quality_scores_category[idx_category]["hard_queries_and_queries_sim_score"]),
        "consistent_score":float(dict_quality_scores_category[idx_category]["consistent_score"]),
        "consistent_hit":float(dict_quality_scores_category[idx_category]["consistent_hit"]),
        "user_intent_score":float(dict_quality_scores_category[idx_category]["user_intent_score"]),
    }
  elif cursor == search: ## Update
    cat["hard_queries_and_queries_sim_score"] = float(dict_quality_scores_category[idx_category]["hard_queries_and_queries_sim_score"])
    cat["consistent_score"] = float(dict_quality_scores_category[idx_category]["consistent_score"])
    cat["consistent_hit"] = float(dict_quality_scores_category[idx_category]["consistent_hit"])
    cat["user_intent_score"] = float(dict_quality_scores_category[idx_category]["user_intent_score"])
    break
if cursor != search:
  new_record = {
        "category":idx_category,
        "hard_queries_and_queries_sim_score": float(dict_quality_scores_category[idx_category]["hard_queries_and_queries_sim_score"]),
        "consistent_score": float(dict_quality_scores_category[idx_category]["consistent_score"]),
        "consistent_hit": float(dict_quality_scores_category[idx_category]["consistent_hit"]),
        "user_intent_score":float(dict_quality_scores_category[idx_category]["user_intent_score"]),
  }
  dict_quality_metrics.append(
      new_record
  )

## Save new file into json
with open(f"/content/data/datasets/metrics/quality/quality_metrics_per_category.json","w+") as f:
    json.dump(dict_quality_metrics,f,indent=2)
    f.close()

## Save file in hugging faces
upload_file(
    path_or_fileobj=f"/content/data/datasets/metrics/quality/quality_metrics_per_category.json",
    path_in_repo=f"datasets/metrics/quality/quality_metrics_per_category.json",
    repo_id="Talissa/AmazonC4Augmented",
    repo_type="dataset",)

## Save dataframe with metrics for hard negative queries in local and in hugging faces
try:
  base = pathlib.Path("/content/data/datasets/procesed/metrics/metrics_for_generated_queries")
  if not base.exists():
    pathlib.Path.mkdir(base,parents=True)
  for i,df_review_metrics_w_scores_batch in enumerate(df_reviews_w_metrics.to_batches()):
    pq.write_table(
        pa.Table.from_pylist(df_review_metrics_w_scores_batch.to_pylist()),
        f"/content/data/datasets/procesed/metrics/metrics_for_generated_queries/{idx_category}_part_{i}.parquet",
    )

    print(f"Saved file in local dir:/content/data/datasets/procesed/metrics/metrics_for_generated_queries/{idx_category}_part_{i}.parquet")

  files = list(base.glob(f"{idx_category}_part_*.parquet"))
  for i,f in enumerate(files):
    upload_file(
        path_or_fileobj=f"{f}",
        path_in_repo=f"/datasets/procesed/metrics/metrics_for_generated_queries/{idx_category}_part_{i}.parquet",
        repo_id="Talissa/AmazonC4Augmented",
        repo_type="dataset",
    )

    print(f"Saved file in remote dir: /datasets/procesed/metrics/metrics_for_generated_queries/{idx_category}_part_{i}.parquet")
except Exception as e:
  print(f"Error for {idx_category}: {e}")

save metrics for All_Beauty
Saved file in local dir:/content/data/datasets/procesed/metrics/metrics_for_generated_queries/All_Beauty_part_0.parquet


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...All_Beauty_part_0.parquet: 100%|##########| 27.2kB / 27.2kB            

Saved file in remote dir: /datasets/procesed/metrics/metrics_for_generated_queries/All_Beauty_part_0.parquet


In [53]:
## Remove all files that are inside the /data directory
import shutil
shutil.rmtree('/content/data')
print("✅ Removed all files that are inside the /data directory")

✅ Removed all files that are inside the /data directory


In [ ]:
# Remove hard negative queries file

In [ ]:
# # Force garbage collection to free CPU memory
# import gc
# gc.collect()

# # Clear GPU cache to free up CUDA memory
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("✅ GPU cache cleared")
# else:
#     print("CUDA not available")

# print("✅ Cleanup complete - model/tokenizer removed from CPU/GPU")

✅ GPU cache cleared
✅ Cleanup complete - model/tokenizer removed from CPU/GPU
